In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

'''  
ydl: means that it is y (the output of nn), but it is just the decay length
_dlc: dlc is for decay length cut

                 0       1     2       3         4           5        6        7       8       9       10
hasTHETA_CHI2  ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'chi2', 'ndof', 'pdgId']
'hasEPZ'      ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'energy', 'pz', 'pdgId']
'hasMEPZ'      ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'mass', 'energy', 'pz', 'pdgId']
In this file, the -1 element (pdgId) is converted to eta, calculated from theta. 

'''

"  \nydl: means that it is y (the output of nn), but it is just the decay length\n_dlc: dlc is for decay length cut\n\n                 0       1     2       3         4           5        6        7       8       9       10\nhasTHETA_CHI2  ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'chi2', 'ndof', 'pdgId']\n'hasEPZ'      ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'energy', 'pz', 'pdgId']\n'hasMEPZ'      ['pt', 'eta', 'phi', 'charge', 'POCA_x', 'POCA_y', 'POCA_z', 'mass', 'energy', 'pz', 'pdgId']\nIn this file, the -1 element (pdgId) is converted to eta, calculated from theta. \n\n"

In [5]:
'''
80/20/20 split train/test/val.
'''
#setting some parameters and strings which are used in the filenames
#these are presented as arrays so that you can do a for loop over them
pTcut = 1.0
ptc = "pTcut:" + str(pTcut) + "GeV"
iserr = "noErr"
#beware that if you make a for loop over opts, the procedure for 'hasTHETA_CHI2_dlCut' is quite different
#so that would be tricky. Instead I have just named mepz and changed its name before doing the decay length cuts
opts = ['hasTHETA_CHI2', 'hasTHETA_CHI2_dlCut', 'hasEPZ', 'hasMEPZ']
scal = "scal"
poca = "pocas"

mepz = 'hasTHETA_CHI2'
print(mepz)
train_frac = 0.6
test_frac = 0.2

PFC_data_np = np.load('PFC_data_%s_%s.npy'%(mepz, poca))
print(PFC_data_np.shape)
SV_true_np = np.load('SV_true_%s_%s.npy'%(mepz, poca))
decay_len_np = np.load('decay_len_%s_%s.npy'%(mepz, poca))

nev = PFC_data_np.shape[0] # no. events
nvar = PFC_data_np.shape[2] # no. vars

#Cuts
for j in range (PFC_data_np.shape[0]):
    for k in range(PFC_data_np.shape[1]):
        #Cut on charge, pdgId, pT)
        if (PFC_data_np[j,k,3] == 0 or PFC_data_np[j,k,-1] == 0 or PFC_data_np[j,k,0] < pTcut): 
            PFC_data_np[j,k] = np.zeros(PFC_data_np.shape[-1])

#find max no. of particles in a jet after this cut
maxp = 0
minp = 100
for j in range (PFC_data_np.shape[0]):
    nump = 0
    for k in range(PFC_data_np.shape[1]):
        if (PFC_data_np[j,k,0] != 0): #this means we have an actual particle, not zero padding
            nump += 1
            #convert pdgId to theta using theta = 2*atan(exp(-eta))
            PFC_data_np[j,k,-1] = 2 * np.arctan(np.exp(-1*PFC_data_np[j,k,1]))
        if (PFC_data_np[j,k,0] < 0 or (PFC_data_np[j,k,0] < 1 and PFC_data_np[j,k,0] != 0)):
            print(j, k, "We have a pT problem!")
    if (nump > maxp): 
        maxp = nump
    if (nump < minp):
        minp = nump
    #order in pT so the all-zero particles will always be the ones lost when i cull
    PFC_data_np[j] = PFC_data_np[j, np.flip((PFC_data_np[j,:,0].argsort()), 0)]




print("maxp (after three cuts): ", maxp)    
print("minp:", minp)
PFC_data_np_new = PFC_data_np[:,:maxp,:]
maxval = np.amax(PFC_data_np_new)
maxind = np.where(PFC_data_np_new == maxval)
minval = np.amin(PFC_data_np_new)
minind = np.where(PFC_data_np_new == minval)
print("max, inds", maxval, maxind)
print("min: ", minval, minind)

print("First two entries pre-scale:", PFC_data_np_new[:2,:,:])

#Apply Scaling
scaler = StandardScaler()
arr = PFC_data_np_new.reshape(PFC_data_np_new.shape[0]*PFC_data_np_new.shape[1], PFC_data_np_new.shape[2])
scaler.fit(arr)
arr = scaler.transform(arr)
PFC_data_np_new = arr.reshape(PFC_data_np_new.shape[0], PFC_data_np_new.shape[1], PFC_data_np_new.shape[2])

print("First two entries post-scale:", PFC_data_np_new[:2,:,:])

cut1 = int(nev*train_frac)
cut2 = cut1 + int(nev*test_frac)

X_train_np = PFC_data_np_new[:cut1]
y_train_np = SV_true_np[:cut1]
ydl_train_np = decay_len_np[:cut1]

X_test_np = PFC_data_np_new[cut1:cut2]
y_test_np = SV_true_np[cut1:cut2]
ydl_test_np = decay_len_np[cut1:cut2]

X_validate_np = PFC_data_np_new[cut2:]
y_validate_np = SV_true_np[cut2:]
ydl_validate_np = decay_len_np[cut2:]

X_train = torch.tensor(X_train_np, dtype=torch.float)
y_train = torch.tensor(y_train_np,dtype=torch.float)
ydl_train = torch.tensor(ydl_train_np,dtype=torch.float)

X_test = torch.tensor(X_test_np, dtype=torch.float)
y_test = torch.tensor(y_test_np, dtype=torch.float)
ydl_test = torch.tensor(ydl_test_np, dtype=torch.float)

X_validate = torch.tensor(X_validate_np, dtype=torch.float)
y_validate = torch.tensor(y_validate_np, dtype = torch.float)
ydl_validate = torch.tensor(ydl_validate_np, dtype = torch.float)

torch.save(X_train, 'X_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_train, 'y_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_train, 'ydl_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(X_test, 'X_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_test, 'y_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_test, 'ydl_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(X_validate, 'X_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_validate, 'y_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_validate, 'ydl_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))

#Now make arrays where I cut out all events where the lifetime is less than 5
mepz = 'hasTHETA_CHI2_dlCut'
print(mepz)
#Make an array of event indexes of lifetimes from highest to lowest, ev_ord means event order
ev_ord = np.flip(decay_len_np.argsort(0), 0)
ev_ord = ev_ord.reshape(ev_ord.shape[0])
#Make all arrays in order of this input
PFC_data_np_dlc = PFC_data_np_new[ev_ord]
SV_true_np_dlc = SV_true_np[ev_ord]
decay_len_np_dlc = decay_len_np[ev_ord]
print("evord:", ev_ord)
#find the first index at which it goes below 5
ff = 0 #first five
g = 0 #index
while (decay_len_np_dlc[g] > 5):
    ff = g + 1
    g += 1
print("ff:", ff)
#shorten the input and output data to only up to this point
PFC_data_np_dlc = PFC_data_np_dlc[:ff]
SV_true_np_dlc = SV_true_np_dlc[:ff]
decay_len_np_dlc = decay_len_np_dlc[:ff]
print("shortened PFC: ", PFC_data_np_dlc.shape, PFC_data_np_dlc)
#reshuffle the data
idx = np.arange(ff)
np.random.shuffle(idx)
PFC_data_np_dlc = PFC_data_np_dlc[idx]
SV_true_np_dlc = SV_true_np_dlc[idx]
decay_len_np_dlc = decay_len_np_dlc[idx]
print("shortened and shuffled PFC: ", PFC_data_np_dlc.shape, PFC_data_np_dlc)
#cut the data at the same fractions, 
c1 = int(ff*train_frac)
c2 = c1 + int(ff*test_frac)

X_train_np = PFC_data_np_dlc[:c1]
y_train_np = SV_true_np_dlc[:c1]
ydl_train_np = decay_len_np_dlc[:c1]

X_test_np = PFC_data_np_dlc[c1:c2]
y_test_np = SV_true_np_dlc[c1:c2]
ydl_test_np = decay_len_np_dlc[c1:c2]

X_validate_np = PFC_data_np_dlc[c2:]
y_validate_np = SV_true_np_dlc[c2:]
ydl_validate_np = decay_len_np_dlc[c2:]

#convert to tensor
X_train = torch.tensor(X_train_np, dtype=torch.float)
y_train = torch.tensor(y_train_np,dtype=torch.float)
ydl_train = torch.tensor(ydl_train_np,dtype=torch.float)

X_test = torch.tensor(X_test_np, dtype=torch.float)
y_test = torch.tensor(y_test_np, dtype=torch.float)
ydl_test = torch.tensor(ydl_test_np, dtype=torch.float)

X_validate = torch.tensor(X_validate_np, dtype=torch.float)
y_validate = torch.tensor(y_validate_np, dtype = torch.float)
ydl_validate = torch.tensor(ydl_validate_np, dtype = torch.float)

#save tensors with new name 

torch.save(X_train, 'X_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_train, 'y_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_train, 'ydl_train_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(X_test, 'X_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_test, 'y_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_test, 'ydl_test_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(X_validate, 'X_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(y_validate, 'y_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))
torch.save(ydl_validate, 'ydl_validate_%s_%s_%s_%s_%s.pt'%(iserr, mepz, poca, scal, ptc))


hasTHETA_CHI2
(70191, 75, 10)
maxp (after three cuts):  29
minp: 0
max, inds 3611.0 (array([65799]), array([3]), array([7]))
min:  -150.95005798339844 (array([48942]), array([0]), array([6]))
First two entries pre-scale: [[[ 2.70000000e+01 -5.86870909e-01  2.04605913e+00  1.00000000e+00
    1.82914257e-01  2.01659396e-01  1.26696360e+00  1.40000000e+01
    1.40000000e+01  2.12661793e+00]
  [ 1.33203125e+01 -7.12302029e-01  1.88804519e+00  1.00000000e+00
   -4.33726460e-02  1.13134116e-01  1.51744652e+00  0.00000000e+00
    2.00000000e+01  2.22953300e+00]
  [ 1.11718750e+01 -5.61235368e-01  1.98179793e+00  1.00000000e+00
    8.18332955e-02  1.58565059e-01  1.21789801e+00  1.40000000e+01
    1.40000000e+01  2.10469518e+00]
  [ 6.65234375e+00 -5.44938505e-01  2.09342480e+00  1.00000000e+00
    1.45719278e+00  9.81468558e-01  2.84448206e-01  0.00000000e+00
    2.00000000e+00  2.09060849e+00]
  [ 4.06250000e+00 -5.63432693e-01  2.02369690e+00 -1.00000000e+00
    1.53552204e-01  1.92033201e-

First two entries post-scale: [[[ 5.26323181e+00 -1.00550953e+00  2.16049192e+00  1.91513722e+00
    2.28360130e-01  2.27238084e-01  4.46920272e-01  3.39400787e-01
    2.03026783e+00  2.07050534e+00]
  [ 2.42185645e+00 -1.21944027e+00  1.99386628e+00  1.91513722e+00
   -1.05749054e-01  9.84834011e-02  5.18805255e-01 -8.49958221e-02
    3.13108561e+00  2.19574523e+00]
  [ 1.97560960e+00 -9.61786491e-01  2.09272851e+00  1.91513722e+00
    7.91156216e-02  1.64559969e-01  4.32839156e-01  3.39400787e-01
    2.03026783e+00  2.04382701e+00]
  [ 1.03686851e+00 -9.33991156e-01  2.21043900e+00  1.91513722e+00
    2.10981306e+00  1.36142349e+00  1.64952531e-01 -8.49958221e-02
   -1.71367716e-01  2.02668457e+00]
  [ 4.98938225e-01 -9.65534169e-01  2.13691096e+00 -1.92276196e+00
    1.85007504e-01  2.13237364e-01  4.34149348e-01  3.55268940e+00
    1.66332858e+00  2.04612756e+00]
  [ 1.10703470e-01 -1.19508021e+00  1.99406829e+00 -1.92276196e+00
   -8.75581161e-02  1.15170708e-01  5.05129441e-01  1

hasTHETA_CHI2_dlCut
evord: [43038 30801  3208 ...  5140 22974  3392]
ff: 39449
shortened PFC:  (39449, 29, 10) [[[-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  ...
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]]

 [[ 2.93648407e-02  2.53824401e+00  2.46254243e+00 ... -8.49958221e-02
    2.76414635e+00  2.16162121e-02]
  [-3.44873990e-01 -4.56395206e-03  2.92332740e-03 ... -8.49958221e-02
   -5.38306974e-01 -5.17428470e-01]
  [-3.44873990e-01 -4.563